In [ ]:
pip install pyshp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 1.8 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns

grouping villages by population in mapping python gis

https://towardsdatascience.com/mapping-geograph-data-in-python-610a963d2d7f

https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/how-grouping-analysis-works.htm

https://gis.stackexchange.com/questions/306547/cluster-points-into-evenly-numbered-groups

https://gis.stackexchange.com/questions/123289/grouping-village-points-based-on-distance-and-population-size-using-arcmap/123297#123297

https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html

https://gis.utah.gov/

To create a voting district map without gerrymandering for the state of Texas, you will need to follow several steps:

Define the geographic boundaries of the state of Texas. You can do this using a shapefile or a geojson file.

Determine the total population of the state of Texas. You can find this information from the United States Census Bureau or other reliable sources.

Divide the total population of the state by the number of districts you want to create. In this case, you want to create districts with approximately 750000 people per district, so you will need to divide the total population by 750000.

Use a geographic information system (GIS) tool, such as QGIS or ArcGIS, to create the districts. You can use the "dissolve" function to combine adjacent areas with similar population densities into a single district.

Review the resulting map to ensure that it meets the requirements for a voting district map without gerrymandering. This includes ensuring that the districts are contiguous and that the population of each district is as close to 750000 as possible.

Here is some example code that shows how you can use the PyShp library to read in a shapefile of the state of Texas and calculate the total population:

In [ ]:
pip install fiona

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.6 MB 4.2 MB/s 


In [ ]:
import json

# Define the GeoJSON data
data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [[[-97.5, 26.5], [-97.5, 30.5], [-93.5, 30.5], [-93.5, 26.5], [-97.5, 26.5]]]
            },
            "properties": {
                "district_id": 1,
                "district_name": "District 1"
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [[[-97.5, 26.5], [-97.5, 30.5], [-93.5, 30.5], [-93.5, 26.5], [-97.5, 26.5]]]
            },
            "properties": {
                "district_id": 2,
                "district_name": "District 2"
            }
        }
    ]
}

# Save the data to a file
with open("texas_voting_districts.geojson", "w") as f:
    json.dump(data, f)

In [ ]:
import fiona

# Read in the GeoJSON file
with open("texas_voting_districts.geojson") as f:
    data = json.load(f)

# Define the shapefile schema
schema = {
    'geometry': 'Polygon',
    'properties': {
        'district_id': 'int',
        'district_name': 'str'
    }
}

# Create the shapefile
with fiona.open("texas_voting_districts.shp", "w", "ESRI Shapefile", schema) as shapefile:
    # Loop through the features in the GeoJSON file and add them to the shapefile
    for feature in data['features']:
        shapefile.write(feature)

# Plots shapefile
def plot_map(sf, x_lim = None, y_lim = None, figsize = (11,9)):
    '''
    Plot map with lim coordinates
    '''
    plt.figure(figsize = figsize)
    id=0
    for shape in sf.shapeRecords():
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        plt.plot(x, y, 'k')
        
        if (x_lim == None) & (y_lim == None):
            x0 = np.mean(x)
            y0 = np.mean(y)
            plt.text(x0, y0, id, fontsize=10)
        id = id+1
    
    if (x_lim != None) & (y_lim != None):     
        plt.xlim(x_lim)
        plt.ylim(y_lim)

plot_map(shp)


AttributeError: ignored

<Figure size 792x648 with 0 Axes>

In [ ]:
num_districts = total_population / population_per_district

# Read in the shapefile using PyShp
sf = shapefile.Reader("texas.shp")

# Calculate the total population
total_population = 0
for record in sf.records():
  population = record[1]
  total_population += population

print(total_population)

# Calculate the number of districts needed
num_districts = total_population / population_per_district

# Create the districts using the "dissolve" function
districts = sf.dissolve(by="POPULATION", num_parts=num_districts)

# Save the districts to a new shapefile
districts.save("texas_districts.shp")

NameError: ignored

In [ ]:
import folium
import shapefile

# Read in the shapefile using PyShp
sf = shapefile.Reader("texas_voting_districts.shp")

# Create a Folium map centered on the state of Texas
m = folium.Map(location=[31.9686, -99.9018], zoom_start=6)

# Loop through the shapes in the shapefile and add them to the map as polygons
for shape in sf.shapes():
  # Get the coordinates of the polygon
  coords = shape.points

  # Create a Folium Polygon object
  poly = folium.Polygon(coords, color="red", fill=True)

  # Add the polygon to the map
  poly.add_to(m)

# Save the map to an HTML file
m.save("texas_map.html")
